# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3

import nltk

nltk.download(['punkt','wordnet','averaged_perceptron_tagger','stopwords'])

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
conn = sqlite3.connect("/content/drive/MyDrive/disaster respose project/dsrDB.db")

In [15]:
df = pd.read_sql("SELECT * FROM msg_cat",conn)

In [20]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3


import nltk

nltk.download(['punkt','wordnet','averaged_perceptron_tagger','stopwords'])

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

def load_data():
    
    #create an engine and extract data from sql
    print("\n")
    print("Loading data into DataFrame")
    df = pd.read_sql("SELECT * FROM msg_cat",conn)
    
    
    #preparing data
    print()
    print("preparing data...")
    target_cols = df.drop(columns = ['index','id','message','original','genre','genre'])
    for col in target_cols.columns:
        print("unique values in all columns:", df[col].unique(),end="")
    target_cols.related.replace(2,1,inplace=True)
    print("\n")
    print("\n after replacing 2 by 1",target_cols.related.unique())
    
    #Making X and y variables
    X = df.message.values
    y = target_cols.values

    print("returned X, y & df varibales")
    
    return X,y,df

def tokenize(text):
    
    
    tokens = word_tokenize(text)
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = [lemmatizer.lemmatize(word).lower().strip() for word in tokens if word not in stop_words]
    
    
    return clean_tokens

def build_model():

    
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])
    
        #set gridsearch parameters 
    parameters = {
      
        'clf__estimator__n_neighbors': (1,2,3),
        'clf__n_jobs': (-1,1)
    }
    cv = GridSearchCV(pipeline, param_grid=parameters)

    return cv
    
def display_results(cv, df, y_test, y_pred):

    print("\n")
    print("Displaying Results...")
    target_names = [col for col in df.columns if col not in ['index','id', 'message', 'original', 'genre']]
    accuracy = (y_pred == y_test).mean()

    print()    
    print("Accuracy:", accuracy)
    print('\n')
    print("\nBest Parameters:", cv.best_params_)
    
    print('\n Classification Report')
    print(classification_report(y_test, y_pred, target_names=target_names))
    
def main():
    X, y, df = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size =.2, random_state=42)
    
    print("\n")
    print("Started tokenizing...")
    print("\n")
    print("Cleaning tokens...")
    model = build_model()

    print("\n")
    print("Returned Clean Tokens")
    print("\n")
    print("Started Training Models...")
    
    model.fit(X_train, y_train)

    print("\n")
    print("Model Returned...")
    print("\n")
    print("Predicting Model...")
    y_pred = model.predict(X_test)

    display_results(model, df, y_test, y_pred)

main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Loading data...

preparing data...
unique values in all columns: [1 0 2]unique values in all columns: [0 1]unique values in all columns: [0 1]unique values in all columns: [0 1]unique values in all columns: [0 1]unique values in all columns: [0 1]unique values in all columns: [0 1]unique values in all columns: [0 1]unique values in all columns: [0 1]unique values in all columns: [0]unique values in all columns: [0 1]unique values in all columns: [0 1]unique value

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning




Model Return...


Predicting Model...


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning




Displaying Results...

Accuracy: 0.9298086702262903



Best Parameters: {'clf__estimator__n_neighbors': 2, 'clf__n_jobs': -1}

 Classification Report
                        precision    recall  f1-score   support

               related       0.80      0.95      0.87      3978
               request       0.84      0.08      0.15       895
                 offer       0.00      0.00      0.00        26
           aid_related       0.81      0.04      0.08      2131
          medical_help       0.33      0.00      0.00       422
      medical_products       1.00      0.01      0.02       270
     search_and_rescue       0.00      0.00      0.00       127
              security       0.00      0.00      0.00        88
              military       0.00      0.00      0.00       155
           child_alone       0.00      0.00      0.00         0
                 water       0.75      0.04      0.08       339
                  food       0.87      0.07      0.12       595
               

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.